# LLAMA3 70B ile yazım hatalarını düzeltme

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq


os.environ["GROQ_API_KEY"] = "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"


client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


base_dir = "/content/NLP-Teknofest24/2000raporBIRADSgruplanmНЯ"


corrected_base_dir = base_dir + "(temizVeri)"

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()


            completion = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {
                        "role": "system",
                        "content": "Verilen raporu kontrol et. Eğer raporda yazım hatası varsa, hataları düzelt ve raporu yaz. Eğer yazım hatası yoksa, raporu olduğu gibi yaz. Sadece yazım hatalarına odaklan başka şeyleri değiştirme. Asla BIRADS Skorunu değiştirme."
                    },
                    {
                        "role": "user",
                        "content": content
                    }
                ],
                temperature=0.1,
                max_tokens=1024,
                top_p=1,
                stream=True,
                stop=None,
            )

            corrected_content = ""
            for chunk in completion:
                corrected_content += chunk.choices[0].delta.content or ""


            relative_path = os.path.relpath(root, base_dir)
            corrected_dir = os.path.join(corrected_base_dir, relative_path)
            if not os.path.exists(corrected_dir):
                os.makedirs(corrected_dir)


            corrected_file_path = os.path.join(corrected_dir, file)
            with open(corrected_file_path, 'w', encoding='utf-8') as f:
                f.write(corrected_content)

print("Tüm dosyalar başarıyla düzeltildi ve kaydedildi.")


In [ ]:
from google.colab import drive
import shutil



source_dir = '/content/NLP-Teknofest24/NLPTXT-1028 Etiketlenen Raporlar(duzeltilmis)'
target_dir = '/content/drive/MyDrive/TeknofestYZ/NLP-TemizlenmisVeri'

if os.path.exists(target_dir):
    target_dir += '_copy'


shutil.copytree(source_dir, target_dir)

print(f"Dizin başarıyla kopyalandı: {target_dir}")


In [ ]:
import os


corrected_base_dir = "/content/NLP-Teknofest24/NLPTXT-1028 Etiketlenen Raporlar(duzeltilmis)"


def clean_report(content):
    unwanted_phrases = [
        "Here is the corrected report:",
        "Corrected errors:",
        "Corrected Errors:",
        "HERE IS THE CORRECTED REPORT:",
        "CORRECTED ERRORS:"
    ]
    lines = content.split("\n")
    cleaned_lines = []

    skip_section = False
    for line in lines:

        if any(phrase in line for phrase in unwanted_phrases):
            skip_section = True
        elif skip_section:
            continue
        else:
            cleaned_lines.append(line)

    return "\n".join(cleaned_lines).strip()


for root, dirs, files in os.walk(corrected_base_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()


            cleaned_content = clean_report(content)


            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(cleaned_content)

print("Tüm dosyalar başarıyla kontrol edildi ve temizlendi.")


In [ ]:
import os
import re


base_dir = "/content/Veri3"


def clean_report(content):
    unwanted_phrases = [
        "Here is the corrected report: ",
        "Here is the corrected report:",
        "Corrected errors:",
        "Corrected Errors:",
        "HERE IS THE CORRECTED REPORT:",
        "CORRECTED ERRORS:",
        "Corrected errors:"
    ]
    lines = content.split("\n")
    cleaned_lines = []
    skip_line = False


    birads_count = content.count("SONUÇ: BIRADS")
    birads_tani_count = content.count("RADYOLOJİK TANI : BİRADS")
    if birads_count > 1 or birads_tani_count > 1:
        return content, True

    for line in lines:
        if any(phrase in line for phrase in unwanted_phrases):
            continue


        if re.match(r"^(SONUÇ: BIRADS \d+|RADYOLOJİK TANI : BİRADS -?\d+)", line):
            cleaned_lines.append(line)
            break

        cleaned_lines.append(line)

    return "\n".join(cleaned_lines).strip(), False

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            cleaned_content, has_multiple_birads = clean_report(content)

            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(cleaned_content)


            if has_multiple_birads:
                print(f"Çoklu 'SONUÇ: BIRADS' veya 'RADYOLOJİK TANI : BİRADS' ifadesi bulunan dosya: {file_path}")
            else:

                birads_pattern = re.compile(r"^(SONUÇ: BIRADS \d+|RADYOLOJİK TANI : BİRADS -?\d+)", re.MULTILINE)
                match = birads_pattern.search(cleaned_content)
                if match:
                    birads_index = match.end()
                    remaining_content = cleaned_content[birads_index:].strip()
                    if remaining_content:
                        print(f"BIRADS sonrası içerik bulunan dosya: {file_path}")

print("Tüm dosyalar başarıyla kontrol edildi ve temizlendi.")


# JSON TEMIZLEME

In [ ]:
import os
import json
from groq import Groq

# API anahtarını ayarlayın
os.environ["GROQ_API_KEY"] = "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"

# Groq istemcisini oluşturun
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Ana dizin
base_dir = "/content/NLP-Teknofest24/NLPJSON"

# Yeni dosyalar için kök dizin
corrected_base_dir = base_dir + "(temizGERCEKVeriJSONll)"

# Ana dizindeki tüm alt klasörleri ve dosyaları gezin
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".jsonl"):
            file_path = os.path.join(root, file)
            corrected_lines = []

            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    json_obj = json.loads(line)
                    text_content = json_obj.get("text", "")

                    # Groq API çağrısı ile dosya içeriğini düzelt
                    completion = client.chat.completions.create(
                        model="llama3-70b-8192",
                        messages=[
                            {
                                "role": "system",
                                "content": "Verilen metindeki yazım hatalarını kontrol et ve düzelt. Yazım hatalarından başka hiçbir şeye odaklanma. Başka hiçbir şey ekleme, sadece yazım hatalarına odaklan. Verinin değişmemesi lazım, bu yüzden bu çok önemli. Ayrıca asla BIRADS Skorunu değiştirme varsayılan neyse o kalsın."
                            },
                            {
                                "role": "user",
                                "content": text_content
                            }
                        ],
                        temperature=0.1,
                        max_tokens=1024,
                        top_p=1,
                        stream=True,
                        stop=None,
                    )

                    corrected_content = ""
                    for chunk in completion:
                        corrected_content += chunk.choices[0].delta.content or ""

                    # Düzeltmiş içeriği JSON nesnesine güncelle
                    json_obj["text"] = corrected_content
                    corrected_lines.append(json.dumps(json_obj, ensure_ascii=False))

            # Yeni klasör yapısını oluştur
            relative_path = os.path.relpath(root, base_dir)
            corrected_dir = os.path.join(corrected_base_dir, relative_path)
            if not os.path.exists(corrected_dir):
                os.makedirs(corrected_dir)

            # Düzeltilmiş dosyayı yeni bir dosyaya kaydet
            corrected_file_path = os.path.join(corrected_dir, file)
            with open(corrected_file_path, 'w', encoding='utf-8') as f:
                for line in corrected_lines:
                    f.write(line + "\n")

print("Tüm dosyalar başarıyla düzeltildi ve kaydedildi.")


In [ ]:
from google.colab import drive

# Google Drive'ı bağlama
drive.mount('/content/drive')

# Kaynak ve hedef dizinler
source_dir = '/content/drive/MyDrive/TeknofestYZ/NLP-TemizlenmisVeri__2'
target_dir = '/content/drive/MyDrive/TeknofestYZ/NLP - Temizlenmis Veriler (egitim icin kullanılacak olan)'

# Kaynak dizini hedef dizine kopyalama (recursive ve interactive)
!cp -r -n "{source_dir}"/* "{target_dir}/"

print(f"Dizin başarıyla kopyalandı: {target_dir}")


In [ ]:
ssource_dirr = '/content/drive/MyDrive/TeknofestYZ/NLP - Temizlenmis Veriler (egitim icin kullanılacak olan)'
main_dir = '/content/egitim'

!cp -r -n "{ssource_dirr}"/* "{main_dir}/"

print(f"Dizin başarıyla kopyalandı: {main_dir}")

# BIRADS 1 RAPOR COGALTMA

In [ ]:
import os
from groq import Groq


os.environ["GROQ_API_KEY"] = "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"


client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


source_dir = "/content/egitim/2000raporBIRADSgruplanmış/BIRADS1reports(n=50)"


generated_base_dir = "/content/generated_reports"


total_reports = 350
current_count = 0

source_files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.endswith('.txt')]


while current_count < total_reports:
    for file_path in source_files:
        if current_count >= total_reports:
            break

        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()


        completion = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "Verilen rapora benzer yeni bir BIRADS1 raporu üret. Rapordaki kelime kullanımına dikkat et ve aynı türde bir rapor oluştur."
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            temperature=0.7,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )

        generated_content = ""
        for chunk in completion:
            generated_content += chunk.choices[0].delta.content or ""


        if not os.path.exists(generated_base_dir):
            os.makedirs(generated_base_dir)


        generated_file_path = os.path.join(generated_base_dir, f"generated_report_{current_count + 1}.txt")
        with open(generated_file_path, 'w', encoding='utf-8') as f:
            f.write(generated_content)

        current_count += 1

print("Toplam 350 adet BIRADS1 raporu başarıyla üretildi ve kaydedildi.")


Toplam 350 adet BIRADS1 raporu başarıyla üretildi ve kaydedildi.


In [ ]:
import os


directory = "/content/unzipped_reportsss/2000raporBIRADSgruplanmìƒ/BIRADS1reports(n=50)"


for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)


        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()


        if lines:
            lines = lines[1:]


        with open(file_path, 'w', encoding='utf-8') as f:
            f.writelines(lines)

print("Tüm dosyalardaki ilk satırlar başarıyla silindi.")


Tüm dosyalardaki ilk satırlar başarıyla silindi.


In [ ]:
from google.colab import drive
import os
import zipfile




zip_path = '/content/drive/My Drive/2000raporBIRADSgruplanmış.zip'


extract_folder = '/content/unzipped_reportsss'


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
    print("ZIP dosyası başarıyla açıldı.")


ZIP dosyası başarıyla açıldı.


In [ ]:
import os
import random
import shutil

def balance_classes(directory, target_count=400):
    for birads_dir in os.listdir(directory):
        birads_path = os.path.join(directory, birads_dir)
        if os.path.isdir(birads_path):
            reports = [f for f in os.listdir(birads_path) if f.endswith(".txt")]
            report_count = len(reports)

            # Fazla olan raporları rastgele seçip sil
            if report_count > target_count:
                reports_to_remove = random.sample(reports, report_count - target_count)
                for report in reports_to_remove:
                    os.remove(os.path.join(birads_path, report))
                print(f"{birads_dir} klasöründen {len(reports_to_remove)} rapor silindi. Yeni toplam: {target_count}")

            # Az olan raporları çoğalt (opsiyonel, genellikle önerilmez)
            elif report_count < target_count:
                print(f"{birads_dir} klasöründe {target_count - report_count} rapor eksik. Bu raporları manuel olarak çoğaltmanız gerekebilir.")

# Belirtilen klasördeki rapor sayılarını 400'e eşitle
balance_classes("/content/unzipped_reportsss/2000raporBIRADSgruplanmìƒ", target_count=400)


BIRADS5reports(n=550) klasöründen 150 rapor silindi. Yeni toplam: 400
BIRADS4reports(n=550) klasöründen 150 rapor silindi. Yeni toplam: 400
BIRADS3reports(n=450) klasöründen 50 rapor silindi. Yeni toplam: 400


In [ ]:
!pip install transformers[torch] accelerate -U scikit-learn imbalanced-learn googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.2 MB/s eta 0:00:00
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
##logistic
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


def read_text_files(directory):
    text_data = []
    labels = []
    label_map = {"BIRADS1": 0, "BIRADS2": 1, "BIRADS3": 2, "BIRADS4": 3, "BIRADS5": 4}
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".txt"):
                filepath = os.path.join(root, filename)
                with open(filepath, 'r', encoding='utf-8') as file:
                    text_data.append(file.read())
                for label in label_map:
                    if label in root:
                        labels.append(label_map[label])
    return text_data, labels


directory = "/content/unzipped_reportsss/2000raporBIRADSgruplanmìƒ"


texts, labels = read_text_files(directory)


train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, train_labels)


LogisticRegression(max_iter=1000)

In [ ]:

predictions = model.predict(X_test)


accuracy = accuracy_score(test_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, predictions, average='weighted')


report = classification_report(test_labels, predictions, target_names=["BIRADS1", "BIRADS2", "BIRADS3", "BIRADS4", "BIRADS5"])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("\nClassification Report:\n", report)


Accuracy: 0.905
Precision: 0.9053077046827046
Recall: 0.905
F1 Score: 0.9046692588844754

Classification Report:
               precision    recall  f1-score   support

     BIRADS1       0.99      1.00      0.99        90
     BIRADS2       0.87      0.87      0.87        77
     BIRADS3       0.86      0.94      0.90        68
     BIRADS4       0.88      0.85      0.86        82
     BIRADS5       0.91      0.86      0.88        83

    accuracy                           0.91       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.91      0.91      0.90       400



In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, train_labels)


rf_predictions = rf_model.predict(X_test)


rf_accuracy = accuracy_score(test_labels, rf_predictions)
rf_precision, rf_recall, rf_f1, _ = precision_recall_fscore_support(test_labels, rf_predictions, average='weighted')


rf_report = classification_report(test_labels, rf_predictions, target_names=["BIRADS1", "BIRADS2", "BIRADS3", "BIRADS4", "BIRADS5"])

print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest Precision: {rf_precision}")
print(f"Random Forest Recall: {rf_recall}")
print(f"Random Forest F1 Score: {rf_f1}")
print("\nRandom Forest Classification Report:\n", rf_report)


Random Forest Accuracy: 0.9275
Random Forest Precision: 0.9293338518910808
Random Forest Recall: 0.9275
Random Forest F1 Score: 0.927133733141489

Random Forest Classification Report:
               precision    recall  f1-score   support

     BIRADS1       0.99      1.00      0.99        90
     BIRADS2       0.89      0.81      0.84        77
     BIRADS3       0.83      0.96      0.89        68
     BIRADS4       0.95      0.96      0.96        82
     BIRADS5       0.96      0.90      0.93        83

    accuracy                           0.93       400
   macro avg       0.92      0.93      0.92       400
weighted avg       0.93      0.93      0.93       400



In [ ]:
def predict_category_rf(text):

    X_new = vectorizer.transform([text])


    predicted_class_id = rf_model.predict(X_new)[0]
    confidence_scores = rf_model.predict_proba(X_new)[0]


    categories = ["BIRADS1", "BIRADS2", "BIRADS3", "BIRADS4", "BIRADS5"]
    predicted_category = categories[predicted_class_id]
    confidence_score = confidence_scores[predicted_class_id]

    return predicted_category, confidence_score


new_text = "BİLATERAL MAMOGRAFİ İNCELEMESİ: Memeler ACR Tip D heterojen yoğun patternde olup, mamografik duyarlılık düşüktür. Her iki memede dağınık noktasal kalsifikasyonlar izlendi. Sağ meme; üst iç kadranda net sınırları seçilemeyen içeriğinde düşük dansiteli alanlar bulunan fokal asimetrik alan izlendi. Farklı kadranlarda konturları fibroglandüler parankimle örtülü izodens nodüler lezyonlar mevcuttur. Sınırları seçilebilen şüpheli kitle lezyonu veya yapısal bozulma saptanmamıştır. Sağ meme alt dış kadranda amorf patternde bölgesel kalsifiye görünümler saptandı. Sağ aksillada görüntülenebilen kesimde patolojik boyut ve görünümde büyümüş lenf nodu izlenmemiştir. Sol meme; Farklı kadranlarda konturları fibroglandüler parankimle örtülü izodens nodüler lezyonlar mevcuttur. Sınırları seçilebilen şüpheli kitle lezyonu, şüpheli kalsifikasyon, yapısal bozulma ya da asimetri saptanmamıştır. Sol aksillada görüntülenebilen kesimde patolojik boyut ve görünümde büyümüş lenf nodu izlenmemiştir."



predicted_category, confidence_score = predict_category_rf(new_text)
print(f"Tahmin Edilen BI-RADS Kategorisi: {predicted_category} (Güven Skoru: {confidence_score:.4f})")


Tahmin Edilen BI-RADS Kategorisi: BIRADS4 (Güven Skoru: 0.9600)


In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV ile hiperparametre optimizasyonu
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, train_labels)

print(f"En iyi parametreler: {grid_search.best_params_}")
best_rf_model = grid_search.best_estimator_


best_rf_predictions = best_rf_model.predict(X_test)


best_rf_accuracy = accuracy_score(test_labels, best_rf_predictions)
best_rf_precision, best_rf_recall, best_rf_f1, _ = precision_recall_fscore_support(test_labels, best_rf_predictions, average='weighted')


best_rf_report = classification_report(test_labels, best_rf_predictions, target_names=["BIRADS1", "BIRADS2", "BIRADS3", "BIRADS4", "BIRADS5"])

print(f"Best Random Forest Accuracy: {best_rf_accuracy}")
print(f"Best Random Forest Precision: {best_rf_precision}")
print(f"Best Random Forest Recall: {best_rf_recall}")
print(f"Best Random Forest F1 Score: {best_rf_f1}")
print("\nBest Random Forest Classification Report:\n", best_rf_report)


Fitting 3 folds for each of 108 candidates, totalling 324 fits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1862a7a315e7>", line 13, in <cell line: 13>
    grid_search.fit(X_train, train_labels)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py", line 1018, in fit
    self._run_search(evaluate_candidates)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py", line 1572, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py", line 964, in evaluate_candidates
    out = parallel(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 74, in __call__
    retu

TypeError: object of type 'NoneType' has no len()

In [ ]:
import joblib


joblib.dump(rf_model, 'rf_birads_model.pkl')

joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

# NER  EĞİTİM

In [ ]:
ssource_dirr = '/content/drive/MyDrive/TeknofestYZ/NLP - Temizlenmis Veriler (egitim icin kullanılacak olan)'
main_dir = '/content/egitim'

!cp -r -n "{ssource_dirr}"/* "{main_dir}/"

print(f"Dizin başarıyla kopyalandı: {main_dir}")

Dizin başarıyla kopyalandı: /content/egitim


In [ ]:
import json
import os

def load_jsonl_files(directory_path):
    data = []
    loaded_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.jsonl'):
                file_path = os.path.join(root, file)
                loaded_files.append(file_path)
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        data.append(json.loads(line))
    return data, loaded_files


directory_path = '/content/egitim/NLPJSON'


json_data, loaded_files = load_jsonl_files(directory_path)

# İlk birkaç veriyi kontrol edelim
print("Loaded JSONL files from the following paths:")
for file in loaded_files:
    print(file)

print("\nFirst few data entries:")
print(json_data[:2])


Loaded JSONL files from the following paths:
/content/egitim/NLPJSON/1/all.jsonl
/content/egitim/NLPJSON/3/all.jsonl
/content/egitim/NLPJSON/4/all.jsonl
/content/egitim/NLPJSON/2/all.jsonl

First few data entries:
[{'id': 284, 'text': 'BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır. \nHer iki meme liposkleroz patterninde izlenmektedir. \nHer iki meme belirgin dense yapıdadır.\nSol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.\nHer iki aksillada patolojik lenf bezi izlenmemektedir.\nSONUÇ: BIRADS 5', 'label': [[32, 47, 'ANAT'], [48, 59, 'ANAT'], [60, 66, 'OBS-PRESENT'], [96, 106, 'OBS-ABSENT'], [110, 121, 'OBS-ABSENT'], [139, 151, 'ANAT'], [152, 165, 'OBS-PRESENT'], [165, 173, 'OBS-PRESENT'], [190, 202, 'ANAT'], [203, 216, 'OBS-PRESENT'], [217, 226, 'OBS-PRESENT'], [228, 238, 'ANAT'], [239, 255, 'ANAT'], [255, 273, 'ANAT'], [273, 287, 'OBS-PRESENT'], [287, 303, 

In [ ]:
!pip install spacy
!pip install spacy-transformers
!python -m spacy download tr_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4

✘ No compatible package found for 'tr_core_news_sm' (spaCy v3.7.6)



In [ ]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random


def convert_to_spacy_format(json_data):
    spacy_data = []
    for entry in json_data:
        text = entry['text']
        entities = []
        for start, end, label in entry['label']:
            entities.append((start, end, label))
        spacy_data.append((text, {"entities": entities}))
    return spacy_data

spacy_data = convert_to_spacy_format(json_data)

# İlk birkaç veriyi kontrol edelim
print(spacy_data[:2])


def clean_and_validate_entities(spacy_data):
    cleaned_data = []
    for text, annotations in spacy_data:
        cleaned_entities = []
        for start, end, label in annotations['entities']:
            entity_text = text[start:end]
            # Boşlukları ve noktalama işaretlerini temizle
            clean_entity_text = entity_text.strip()
            clean_start = text.find(clean_entity_text)
            clean_end = clean_start + len(clean_entity_text)
            if clean_start != -1 and clean_end <= len(text):
                cleaned_entities.append((clean_start, clean_end, label))

        # Çakışan varlıkları kaldır
        non_overlapping_entities = []
        for start, end, label in cleaned_entities:
            overlap = False
            for n_start, n_end, n_label in non_overlapping_entities:
                if not (end <= n_start or start >= n_end):
                    overlap = True
                    break
            if not overlap:
                non_overlapping_entities.append((start, end, label))

        cleaned_data.append((text, {"entities": non_overlapping_entities}))
    return cleaned_data


cleaned_spacy_data = clean_and_validate_entities(spacy_data)


print(cleaned_spacy_data[:2])


[('BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır. \nHer iki meme liposkleroz patterninde izlenmektedir. \nHer iki meme belirgin dense yapıdadır.\nSol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.\nHer iki aksillada patolojik lenf bezi izlenmemektedir.\nSONUÇ: BIRADS 5', {'entities': [(32, 47, 'ANAT'), (48, 59, 'ANAT'), (60, 66, 'OBS-PRESENT'), (96, 106, 'OBS-ABSENT'), (110, 121, 'OBS-ABSENT'), (139, 151, 'ANAT'), (152, 165, 'OBS-PRESENT'), (165, 173, 'OBS-PRESENT'), (190, 202, 'ANAT'), (203, 216, 'OBS-PRESENT'), (217, 226, 'OBS-PRESENT'), (228, 238, 'ANAT'), (239, 255, 'ANAT'), (255, 273, 'ANAT'), (273, 287, 'OBS-PRESENT'), (287, 303, 'OBS-PRESENT'), (313, 330, 'ANAT'), (331, 350, 'OBS-ABSENT'), (375, 383, 'IMPRESSION')]}), ('\n\nBİLATERAL MAMOGRAFİ İNCELEMESİNE;\nBilateral meme cilt kalınlığı normaldir.\nHer iki meme tip B patternedir.\nHer iki memede spiküle

In [ ]:

nlp = spacy.blank("tr")


if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")


for _, annotations in cleaned_spacy_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])


print(cleaned_spacy_data[:2])

optimizer = nlp.begin_training()




n_iter = 30
for itn in range(n_iter):
    random.shuffle(cleaned_spacy_data)
    losses = {}

    batches = minibatch(cleaned_spacy_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        examples = []
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        nlp.update(examples, drop=0.5, losses=losses)
    print(f"Iteration {itn} - Losses: {losses}")


nlp.to_disk("ner_model")



/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[('BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır. \nHer iki meme liposkleroz patterninde izlenmektedir. \nHer iki meme belirgin dense yapıdadır.\nSol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.\nHer iki aksillada patolojik lenf bezi izlenmemektedir.\nSONUÇ: BIRADS 5', {'entities': [(32, 47, 'ANAT'), (48, 59, 'ANAT'), (60, 66, 'OBS-PRESENT'), (97, 106, 'OBS-ABSENT'), (111, 121, 'OBS-ABSENT'), (153, 164, 'OBS-PRESENT'), (165, 173, 'OBS-PRESENT'), (137, 149, 'ANAT'), (203, 216, 'OBS-PRESENT'), (217, 226, 'OBS-PRESENT'), (228, 238, 'ANAT'), (239, 255, 'ANAT'), (255, 273, 'ANAT'), (273, 287, 'OBS-PRESENT'), (287, 303, 'OBS-PRESENT'), (313, 330, 'ANAT'), (331, 349, 'OBS-ABSENT'), (375, 383, 'IMPRESSION')]}), ('\n\nBİLATERAL MAMOGRAFİ İNCELEMESİNE;\nBilateral meme cilt kalınlığı normaldir.\nHer iki meme tip B patternedir.\nHer iki memede spiküle konturlu kitle lezy

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "

BİLATERAL MAMOGRAFİ İNCELEMESİ
Her iki meme tip ..." with entities "[(33, 44, 'ANAT'), (46, 73, 'OBS-PRESENT'), (75, 8...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "

BİLATERAL MAMOGRAFİ İNCELEMESİ:
Her iki meme dok..." with entities "[(32, 54, 'ANAT'), (57, 69, 'ANAT'), (70, 75, 'OBS...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Iteration 0 - Losses: {'ner': 8441.02444943982}
Iteration 1 - Losses: {'ner': 6810.502358624192}
Iteration 2 - Losses: {'ner': 5700.988665006243}
Iteration 3 - Losses: {'ner': 5320.581229710741}
Iteration 4 - Losses: {'ner': 5186.055079306444}
Iteration 5 - Losses: {'ner': 5027.020789697992}
Iteration 6 - Losses: {'ner': 4790.366766178219}
Iteration 7 - Losses: {'ner': 4725.31674057546}
Iteration 8 - Losses: {'ner': 4633.563590849093}
Iteration 9 - Losses: {'ner': 4576.2300565061505}
Iteration 10 - Losses: {'ner': 4612.335786175789}
Iteration 11 - Losses: {'ner': 4382.181614734527}
Iteration 12 - Losses: {'ner': 4375.491614044266}
Iteration 13 - Losses: {'ner': 4297.201509758556}
Iteration 14 - Losses: {'ner': 4281.04991576239}
Iteration 15 - Losses: {'ner': 4227.925158775904}
Iteration 16 - Losses: {'ner': 4220.802886002369}
Iteration 17 - Losses: {'ner': 4233.884654093755}
Iteration 18 - Losses: {'ner': 4208.922888411358}
Iteration 19 - Losses: {'ner': 4149.580766303877}
Iteration 20

In [ ]:

nlp2 = spacy.load("ner_model")


test_texts = [
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
    "UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki meme parankimi ACR tip D paterndedir. Dens meme paterni nedeniyle mamografi tanı duyarlılığı yetersiz olup mamogramlar dahilinde belirgin trabeküler distorsiyon veya yer kaplayan lezyon izlenmemiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Sol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi. Her iki aksillada patolojik lenf bezi izlenmemektedir. SONUÇ: BIRADS 5."
]

for test_text in test_texts:
    doc = nlp2(test_text)
    print(f"Test text: {test_text}")
    for ent in doc.ents:
        print(f"Entity: {ent.text}, Start: {ent.start_char}, End: {ent.end_char}, Label: {ent.label_}")
    print("\n")


Test text: BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.
Entity: cilt yüzeyi, Start: 48, End: 59, Label: ANAT
Entity: normal, Start: 60, End: 66, Label: OBS-PRESENT
Entity: kalınlaşma, Start: 97, End: 107, Label: OBS-ABSENT
Entity: retraksiyon, Start: 111, End: 122, Label: OBS-ABSENT


Test text: UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.
Entity: cilt kalınlığı, Start: 42, End: 56, Label: ANAT
Entity: normaldir, Start: 57, End: 66, Label: OBS-PRESENT
Entity: 23 mm çapında, Start: 85, End: 98, Label: OBS-PRESENT
Entity: spiküle konturlu, Start: 99, End: 115, Label: OBS-PRESENT


Test text: BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.
Entity: cilt yüzeyi, Start: 48, End: 59, Label: ANAT
Enti

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

def align_labels(true_entities, pred_entities, text_length):
    true_labels = ["O"] * text_length
    pred_labels = ["O"] * text_length

    for start, end, label in true_entities:
        for i in range(start, end):
            if i == start:
                true_labels[i] = f"B-{label}"
            else:
                true_labels[i] = f"I-{label}"

    for start, end, label in pred_entities:
        for i in range(start, end):
            if i == start:
                pred_labels[i] = f"B-{label}"
            else:
                pred_labels[i] = f"I-{label}"

    return true_labels, pred_labels

def evaluate_model(nlp, data):
    y_true = []
    y_pred = []

    for text, annotations in data:
        doc = nlp(text)
        true_entities = [(start, end, label) for start, end, label in annotations["entities"]]
        pred_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

        true_labels, pred_labels = align_labels(true_entities, pred_entities, len(text))

        y_true.extend(true_labels)
        y_pred.extend(pred_labels)


    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    return precision, recall, f1


test_data = [
    ("BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
     {"entities": [(48, 59, "ANAT"), (60, 66, "OBS-PRESENT"), (97, 107, "OBS-PRESENT")]}),
    ("UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.",
     {"entities": [(42, 56, "ANAT"), (57, 67, "OBS-PRESENT"), (85, 98, "OBS-PRESENT"), (99, 121, "OBS-PRESENT")]}),
]

precision, recall, f1 = evaluate_model(nlp2, test_data)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Precision: 0.9758
Recall: 0.9051
F1 Score: 0.9357


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

nlp2 = spacy.load("ner_model")


precision, recall, f1 = evaluate_model(nlp2, test_data)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Precision: 0.9758
Recall: 0.9051
F1 Score: 0.9357


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ISTEDIKLERI FORMAT

In [ ]:
import spacy


nlp2 = spacy.load("ner_model")


test_texts = [
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
    "UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki meme parankimi ACR tip D paterndedir. Dens meme paterni nedeniyle mamografi tanı duyarlılığı yetersiz olup mamogramlar dahilinde belirgin trabeküler distorsiyon veya yer kaplayan lezyon izlenmemiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Sol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi. Her iki aksillada patolojik lenf bezi izlenmemektedir. SONUÇ: BIRADS 5."
]


test_results = []


for i, test_text in enumerate(test_texts):
    doc = nlp2(test_text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "start": ent.start_char,
            "end": ent.end_char,
            "label": ent.label_
        })


    test_results.append({
        "id": i + 1,
        "text": test_text,
        "entities": entities
    })


import json
print(json.dumps(test_results, ensure_ascii=False, indent=4))


[
    {
        "id": 1,
        "text": "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
        "entities": [
            {
                "start": 48,
                "end": 59,
                "label": "ANAT"
            },
            {
                "start": 60,
                "end": 66,
                "label": "OBS-PRESENT"
            },
            {
                "start": 97,
                "end": 107,
                "label": "OBS-ABSENT"
            },
            {
                "start": 111,
                "end": 122,
                "label": "OBS-ABSENT"
            }
        ]
    },
    {
        "id": 2,
        "text": "UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.",
        "entities": [
            {
                "start": 42,
                "end": 5

In [ ]:
import spacy
import joblib
import json


nlp2 = spacy.load("ner_model")


rf_model = joblib.load('rf_birads_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')


In [ ]:

test_texts = [
    "Bilateral meme cilt kalınlığı normaldir. Her iki meme tip B patternindedir. Her iki memede spiküle konturlu kitle lezyonu, patolojik kalsifikasyon veya yapısal distorsiyon kaydedilmemiştir. Her iki memede dağınık yerleşimli birkaç adet noktasal ve makrokalsifikasyon izlendi. Sol meme alt iç kadranda kalsiyum sütüne ait görünümler izlendi.",
    "UNİLATERAL MAMOGRAFİ İNCELEMESİ: Sol meme cilt kalınlığı normaldir. Üst dış kadranda 23 mm çapında spiküle konturlu kitle ile uyumlu radyopasite izlenmiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup, kalınlaşma ve retraksiyon saptanmamıştır.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Her iki meme parankimi ACR tip D paterndedir. Dens meme paterni nedeniyle mamografi tanı duyarlılığı yetersiz olup mamogramlar dahilinde belirgin trabeküler distorsiyon veya yer kaplayan lezyon izlenmemiştir.",
    "BİLATERAL MAMOGRAFİ İNCELEMESİ: Memeler ACR Tip C heterojen yoğun pattern'da olup, mamografik duyarlılık düşüktür. Her iki memede dağınık yerleşimli noktasal kalsifikasyonlar mevcuttur. Sol meme; üst iç kadranda silik sınırlı nodüler opasite mevcuttur. MLO grafi üst kadranda nispeten yüzeyel yerleşimli çok düşük dansiteli nodüler fokal asimetrik görünüm mevcuttur. CC grafi iç kadranda düzgün konturlu düşük dansiteli nodüler fokal asimetri mevcuttur. Sınırları seçilebilen şüpheli kitle lezyonu, yapısal bozulma saptanmamıştır. Sol meme üst dış kadranda amorf pattern'de kalsifikasyonlar saptandı. Sol aksillada görüntülenebilen kesimde patolojik boyut ve görünümde büyümüş lenf nodu izlenmemiştir. Sağ meme; üst dış kadranda derin yerleşimli sınırları net seçilemeyen fokal asimetrik alan mevcuttur. Sınırları seçilebilen şüpheli kitle lezyonu, şüpheli kalsifikasyon, yapısal bozulma saptanmamıştır. Sağ aksillada görüntülenebilen kesimde patolojik boyut ve görünümde büyümüş lenf nodu izlenmemiştir."
]


test_results = []


def predict_category_rf(text):

    X_new = vectorizer.transform([text])


    predicted_class_id = rf_model.predict(X_new)[0]
    confidence_scores = rf_model.predict_proba(X_new)[0]


    categories = ["BIRADS1", "BIRADS2", "BIRADS3", "BIRADS4", "BIRADS5"]
    predicted_category = categories[predicted_class_id]
    confidence_score = confidence_scores[predicted_class_id]

    return predicted_category, confidence_score


for i, test_text in enumerate(test_texts):

    doc = nlp2(test_text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "start": ent.start_char,
            "end": ent.end_char,
            "label": ent.label_
        })


    predicted_category, confidence_score = predict_category_rf(test_text)


    test_results.append({
        "id": i + 1,
        "text": test_text,
        "cats": predicted_category,
        "confidence": round(confidence_score, 4),
        "entities": entities
    })

print(json.dumps(test_results, ensure_ascii=False, indent=4))


[
    {
        "id": 1,
        "text": "Bilateral meme cilt kalınlığı normaldir. Her iki meme tip B patternindedir. Her iki memede spiküle konturlu kitle lezyonu, patolojik kalsifikasyon veya yapısal distorsiyon kaydedilmemiştir. Her iki memede dağınık yerleşimli birkaç adet noktasal ve makrokalsifikasyon izlendi. Sol meme alt iç kadranda kalsiyum sütüne ait görünümler izlendi.",
        "cats": "BIRADS2",
        "confidence": 0.48,
        "entities": [
            {
                "start": 0,
                "end": 14,
                "label": "ANAT"
            },
            {
                "start": 15,
                "end": 29,
                "label": "ANAT"
            },
            {
                "start": 30,
                "end": 39,
                "label": "OBS-PRESENT"
            },
            {
                "start": 54,
                "end": 59,
                "label": "OBS-PRESENT"
            },
            {
                "start": 91,
              